In [4]:
import pandas as pd
import numpy as np

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import re
from pandas.testing import assert_series_equal, assert_frame_equal

In [661]:
pd.options.display.max_rows = 999

## Comparing files

In [95]:
z = pd.read_excel('new_anomalies_compare_FV_270119.xlsx', sheet_name='Paying Accounts', use_cols=3)
paying_accs = z['ID'].unique()
len(paying_accs)

343

In [97]:
anom_new[(anom_new.pai_account_id.isin(paying_accs)) & (anom_new.critical_flag == 1)].pai_account_id.unique()

array(['0010J00001osBe1', '0010J00001pG2vr', '001b00000067QDW',
       '001b0000006AArd', '001b000000S94sv', '001b000000XqAnZ',
       '001b000000Ywov7', '001b000000Ztyh2', '001b000000c5Oef',
       '001b000000eiCtG', '001b000000fTU6Q', '001b000000iB3RJ',
       '001b000000n1pvG', '001b000000qH8z9', '001b000000rHLNw',
       '001b000000sB52F', '001b000001EhSOT', '001b000001EjK7y',
       '001b000003ODKDn', '001b000003UuaRn', '001b000003gqElA',
       '001b000003grgyD', '001b000003qm4Ej', '001b000003xeKAI'],
      dtype=object)

In [181]:
proc_new = pd.read_csv('data/processed/new.csv')
proc_old = pd.read_csv('data/processed/old.csv')
anom_new = pd.read_csv('data/anomalies/new.csv')
anom_old = pd.read_csv('data/anomalies/old.csv')

In [169]:
cleaned = pd.read_csv('data/cleaned.csv', dtype='object')

In [182]:
proc_old.shape, proc_new.shape

((501107, 7), (514795, 7))

In [183]:
proc_new = proc_new.sort_values(by=['pai_account_id', 'date'])
proc_old = proc_old.sort_values(by=['pai_account_id', 'date'])

In [184]:
shared_ids = set(proc_new.pai_account_id).union(proc_old.pai_account_id)
proc_new = proc_new[proc_new.pai_account_id.isin(shared_ids)]
proc_old = proc_old[proc_old.pai_account_id.isin(shared_ids)]

In [192]:
set(proc_new.churn_flag).intersection(proc_new.client_id)

{1}

In [185]:
shared_dates = set(proc_old.date).union(proc_new.date)
proc_new = proc_new[proc_new.date.isin(shared_dates)]
proc_old = proc_old[proc_old.date.isin(shared_dates)]

In [186]:
proc_old.shape, proc_new.shape

((501107, 7), (514795, 7))

In [187]:
proc_old = proc_old.set_index(['pai_account_id', 'date', 'stage'])
proc_new = proc_new.set_index(['pai_account_id', 'date', 'stage'])

In [188]:
inter_ind = proc_old.index.intersection(proc_new.index)

In [189]:
proc_old = proc_old.loc[inter_ind]
proc_new = proc_new.loc[inter_ind]

In [190]:
proc_old.shape, proc_new.shape

((501107, 4), (501107, 4))

In [191]:
assert_frame_equal(proc_old, proc_new)

In [638]:
test_new = proc_new.loc[inter_ind]
test_old = proc_old.loc[inter_ind]

In [640]:
test_new.shape, test_old.shape

((437387, 5), (437387, 5))

In [648]:
joined = pd.concat([test_old, test_new], axis=1)

Churn flag:

In [651]:
joined[joined.iloc[:,0] != joined.iloc[:,5]]

churn_flag  client_id     profile stage  \
pai_account_id  date                                                  
001b000002LQE31 2016-02-12           1          1  enterprise   poc   
                2016-03-14           0          1  enterprise   poc   

                           stage_start_date  churn_flag  client_id  \
pai_account_id  date                                                 
001b000002LQE31 2016-02-12       2016-01-13           0          1   
                2016-03-14       2016-01-13           1          1   

                               profile stage stage_start_date  
pai_account_id  date                                           
001b000002LQE31 2016-02-12  enterprise   poc       2016-01-13  
                2016-03-14  enterprise   poc       2016-01-13

It's a bad account - was poc all its life. In new it's churned on last day, in old it churns in middle then goes back to 0!

In [662]:
joined.loc['001b000002LQE31'].head(90)

,churn_flag,client_id,profile,stage,stage_start_date,churn_flag,client_id,profile,stage,stage_start_date
date,,,,,,,,,,
2016-01-13,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-14,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-15,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-16,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-17,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-18,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-19,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-20,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-21,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13


In [666]:
cleaned[cleaned.pai_account_id == '001b000002LQE31'].churn_in_stage.value_counts()

1    95
Name: churn_in_stage, dtype: int64

Client_id, profile(!):

In [652]:
joined[joined.iloc[:,1] != joined.iloc[:,6]]

,,churn_flag,client_id,profile,stage,stage_start_date,churn_flag,client_id,profile,stage,stage_start_date
pai_account_id,date,,,,,,,,,,


In [653]:
joined[joined.iloc[:,2] != joined.iloc[:,7]]

,,churn_flag,client_id,profile,stage,stage_start_date,churn_flag,client_id,profile,stage,stage_start_date
pai_account_id,date,,,,,,,,,,


Stage (1503):

In [667]:
temp = joined[joined.iloc[:,3] != joined.iloc[:,8]]
temp

churn_flag  client_id     profile   stage  \
pai_account_id  date                                                    
001b000000c5VoN 2017-03-19           0          1  enterprise  paying   
001b000000p20V8 2017-04-16           0          1         smb  paying   
                2017-04-17           0          1         smb  paying   
                2017-04-18           0          1         smb  paying   
                2017-04-19           0          1         smb  paying   
                2017-04-20           0          1         smb  paying   
                2017-04-21           0          1         smb  paying   
                2017-04-22           0          1         smb  paying   
                2017-04-23           0          1         smb  paying   
                2017-04-24           0          1         smb  paying   
                2017-04-25           0          1         smb  paying   
                2017-04-26           0          1         smb  paying   
                2017-04-27           0          1         smb  paying   
                2017-04-28           0          1         smb  paying   
                2017-04-29           0          1         smb  paying   
                2017-04-30           0          1         smb  paying   
                2017-05-01           0          1         smb  paying   
                2017-05-02           0          1         smb  paying   
                2017-05-03           0          1         smb  paying   
                2017-05-04           0          1         smb  paying   
                2017-05-05           0          1         smb  paying   
                2017-05-06           0          1         smb  paying   
                2017-05-07           0          1         smb  paying   
                2017-05-08           0          1         smb  paying   
                2017-05-09           0          1         smb  paying   
                2017-05-10           0          1         smb  paying   
                2017-05-11           0          1         smb  paying   
                2017-05-12           0          1         smb  paying   
                2017-05-13           0          1         smb  paying   
                2017-05-14           0          1         smb  paying   
                2017-05-15           0          1         smb  paying   
                2017-05-16           0          1         smb  paying   
                2017-05-17           0          1         smb  paying   
                2017-05-18           0          1         smb  paying   
                2017-05-19           0          1         smb  paying   
                2017-05-20           0          1         smb  paying   
                2017-05-21           0          1         smb  paying   
                2017-05-22           0          1         smb  paying   
                2017-05-23           0          1         smb  paying   
                2017-05-24           0          1         smb  paying   
                2017-05-25           0          1         smb  paying   
                2017-05-26           0          1         smb  paying   
                2017-05-27           0          1         smb  paying   
                2017-05-28           0          1         smb  paying   
                2017-05-29           0          1         smb  paying   
                2017-05-30           0          1         smb  paying   
                2017-05-31           0          1         smb  paying   
                2017-06-01           0          1         smb  paying   
                2017-06-02           0          1         smb  paying   
                2017-06-03           0          1         smb  paying   
                2017-06-04           0          1         smb  paying   
                2017-06-05           0          1         smb  paying   
                2017-06-06           0          1         smb  paying   
             

All changes are from paying to poc

In [668]:
print(temp.iloc[:,3].value_counts(), temp.iloc[:,8].value_counts())

paying    1503
Name: stage, dtype: int64 poc    1503
Name: stage, dtype: int64


Stage start date (11510!):

This is understandable since if we changed the stage, we change the stage start date all along

In [655]:
joined[joined.iloc[:,4] != joined.iloc[:,9]]

churn_flag  client_id     profile   stage  \
pai_account_id  date                                                    
001b000000c5VoN 2017-03-19           0          1  enterprise  paying   
                2017-03-20           0          1  enterprise  paying   
                2017-03-21           0          1  enterprise  paying   
                2017-03-22           0          1  enterprise  paying   
                2017-03-23           0          1  enterprise  paying   
                2017-03-24           0          1  enterprise  paying   
                2017-03-25           0          1  enterprise  paying   
                2017-03-26           0          1  enterprise  paying   
                2017-03-27           0          1  enterprise  paying   
                2017-03-28           0          1  enterprise  paying   
                2017-03-29           0          1  enterprise  paying   
                2017-03-30           0          1  enterprise  paying   
                2017-03-31           0          1  enterprise  paying   
                2017-04-01           0          1  enterprise  paying   
                2017-04-02           0          1  enterprise  paying   
                2017-04-03           0          1  enterprise  paying   
                2017-04-04           0          1  enterprise  paying   
                2017-04-05           0          1  enterprise  paying   
                2017-04-06           0          1  enterprise  paying   
                2017-04-07           0          1  enterprise  paying   
                2017-04-08           0          1  enterprise  paying   
                2017-04-09           0          1  enterprise  paying   
                2017-04-10           0          1  enterprise  paying   
                2017-04-11           0          1  enterprise  paying   
                2017-04-12           0          1  enterprise  paying   
                2017-04-13           0          1  enterprise  paying   
                2017-04-14           0          1  enterprise  paying   
                2017-04-15           0          1  enterprise  paying   
                2017-04-16           0          1  enterprise  paying   
                2017-04-17           0          1  enterprise  paying   
...                                ...        ...         ...     ...   
001b000003s29Mp 2018-07-14           0          1  enterprise  paying   
                2018-07-15           0          1  enterprise  paying   
                2018-07-16           0          1  enterprise  paying   
                2018-07-17           0          1  enterprise  paying   
                2018-07-18           0          1  enterprise  paying   
                2018-07-19           0          1  enterprise  paying   
                2018-07-20           0          1  enterprise  paying   
                2018-07-21           0          1  enterprise  paying   
                2018-07-22           0          1  enterprise  paying   
                2018-07-23           0          1  enterprise  paying   
                2018-07-24           0          1  enterprise  paying   
                2018-07-25           0          1  enterprise  paying   
                2018-07-26           0          1  enterprise  paying   
                2018-07-27           0          1  enterprise  paying   
                2018-07-28           0          1  enterprise  paying   
                2018-07-29           0          1  enterprise  paying   
                2018-07-30           0          1  enterprise  paying   
                2018-07-31           0          1  enterprise  paying   
                2018-08-01           0          1  enterprise  paying   
                2018-08-02           0          1  enterprise  paying   
                2018-08-03           0          1  enterprise  paying   
                2018-08-04           0          1  enterprise  paying   
             

In [643]:
cols = list(test_new)
for i in range(len(cols)):
    print(cols[i])
    assert_series_equal(test_old.iloc[:,i], test_new.iloc[:,i])

churn_flag


AssertionError: Series are different

Series values are different (0.00046 %)
[left]:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]
[right]: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]

Now let's look at the set diff:

In [669]:
diff_old_ind = proc_old.index.difference(proc_new.index)
len(diff_old_ind)

In [671]:
diff_new_ind = proc_new.index.difference(proc_old.index)
len(diff_new_ind)

16

In [672]:
diff_new_ind

MultiIndex(levels=[['001b000000p20V8', '001b000003hW9ZA', '001b000003j7RxW', '001b000003jzTcQ', '001b000003jzdhk', '001b000003kf21M'], ['2017-04-14', '2017-04-15', '2017-04-21', '2017-04-22', '2017-04-23', '2017-04-24']],
           labels=[[0, 1, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5], [1, 3, 4, 5, 4, 5, 0, 1, 2, 3, 4, 5, 2, 3, 4, 5]],
           names=['pai_account_id', 'date'],
           sortorder=0)

In [673]:
diff_old_ind

MultiIndex(levels=[['0010J00001oc4O9', '0010J00001oc4q3', '0010J00001oc5JA', '0010J00001oc8BY', '0010J00001oc8CR', '0010J00001oc8DF', '0010J00001oc8VO', '0010J00001ooU2z', '0010J00001ooUjf', '0010J00001ooUlg', '0010J00001ooV78', '0010J00001ooVZB', '0010J00001ooX0r', '0010J00001opSZk', '0010J00001opUHM', '0010J00001opZQT', '0010J00001opa7S', '0010J00001oq1Ks', '0010J00001oqE7n', '0010J00001oqgkS', '0010J00001oqpnf', '0010J00001oqsqw', '0010J00001or0lk', '0010J00001or2yM', '0010J00001orC6W', '0010J00001orC7j', '0010J00001os9YZ', '0010J00001osExa', '0010J00001pFQ4e', '0010J00001pFk2v', '0010J00001pFkEw', '0010J00001pFtOl', '0010J00001pG7pF', '0010J00001sOm4D', '0010J00001spJdB', '0010J00001spR4s', '0010J00001spmKU', '0010J00001spvzw', '0010J00001spwSF', '0010J00001sq6eQ', '0010J00001sqGlJ', '0010J00001sqXoG', '0010J00001sqnWX', '0010J00001sqsnY', '0010J00001sqx0M', '0010J00001sr81X', '0010J00001sriLY', '0010J00001xx0Zv', '0010J00001yQjwS', '0010J00001yRutG', '0010J00001yTJxk', '0010J00001